### Реализация и исследование L-BFGS

L-BFGS (L - limited mempry) --- модификация метода BFGS с ограниченной памятью. 
Обычный BFGS хранит приближение к обратному гессиану в виде квадратной неразряженной матрицы и это может потреблять очень большой обьём памяти в случаях, когда число переменных огромное. В таких случаях может быть использован LBFGS.

Идея в том, что вместо целой матрицы хранится ограниченная (числом m) история изменений позиции $\Delta x$ и градиента $\Delta \nabla f(x)$, на основе которых на каждом шагу приближается обратный к гессиану и вычиялется направление поиска. Часто это число m < 20

In [79]:
from matplotlib import pyplot as plt
import numpy as np
import random as rand
from collections import deque
# import bfgs_utils as ut
plt.rcParams["figure.figsize"] = (20,10)

In [80]:
class History:
    s = None # dx
    y = None
    rho = None

    def __init__(self, m = 5):
        self.s = deque([], m)
        self.y = deque([], m)
        self.rho = deque([], m)

    def update(dx, dg):
        self.s.add(dx)
        self.y.add(dg)
        self.rho.add(1. / (np.dot(dg, dx) + 1e-30))

In [ ]:
def wolfe_search(f, x, p, grad, nabla):
    '''
    Поиск с условиями Вольфе
    '''
    a = 1
    c1 = 1e-4 
    c2 = 0.9 
    fx = f(x)
    x_new = x + a * p 
    nabla_new = grad(f, x_new)
    while f(x_new) >= fx + (c1*a*nabla.T @ p) or nabla_new.T @ p <= c2*nabla.T @ p : 
        a *= 0.5
        x_new = x + a * p 
        nabla_new = grad(f, x_new)
    return a

In [81]:
def get_direction(grad, history, h_init):
    q = np.copy(grad)
    m = len(history.rho)
    alpha = np.zeros(m)

    for i in range(m - 1, -1, -1):
        alpha[i] = history.rho[i] * np.dot(history.s[i], q)
        q = q - alpha[i] * history.y[i]

    if m == 0:
        H_init = h_init
        if isinstance(h_init, np.ndarray):
            r = np.matmul(H_init, q.reshape(-1, 1)).T[0]
        else:
            r = H_init * q
    else:
        H_init = np.dot(history.s[-1], history.y[-1]) / (np.dot(history.y[-1], history.y[-1]) + 1e-30)
        r = H_init * q

    for i in range(m):
        r = r + history.s[i] * (alpha[i] - history.rho[i] * np.dot(history.y[i], r))

    return -r

In [89]:
def lbfgs(f, x0, grad, m = 5, eps = 1e-9, epochs=100, h_init=1, **config):
    x = np.copy(x0)
    points_history = [x]
    g = np.array(grad(x))
    history = History(m)

    for epoch in 0..epochs+1:
        if not (np.linalg.norm(g) > eps and epoch <= epochs):
            break

        d = get_direction(g, history, h_init=h_init)
        x_new = wolfe_search(f, x, grad)
        g_new = grad(x_new)
        history.update(x_new - x, g_new - g)
        x = x_new
        g = g_new
        points_history.append(x_new)
        epoch += 1
    return points_history

### Тест

Предлагается тест LBFGS на задаче линейной регрессии с очень большой размерностью.

In [83]:
rand = lambda n: (np.random.rand(n)*2)-1

def random_perfect_line(n, k, k_limit, ms_limit, debug=False):
    m = rand(n) * ms_limit
    s = rand(n) * ms_limit
    ks = rand(k) * k_limit
    xy = np.array([m + s * k for k in ks])
    x = xy[:, :-1]
    y = xy[:, -1]
    # if debug:
    #     print(f'{m=}\n{s=}\n{x=}\n{y=}')
    return x, y

In [84]:
def regression(x, y, method, batch_size=10, **config):
    if config == {}:
        config = {"lr0": 0.5, "d": 0.005, "epoch": 1000}
    x_mat = np.hstack((np.ones((x.shape[0], 1)), x))
    k = x_mat.shape[1]
    batch_choice = lambda batch_size: list(set(np.random.choice(np.arange(x.shape[0]), batch_size, replace=False)))
    f_batch_size = lambda batch_size: \
                       lambda *b, batch=batch_choice(batch_size): \
                           np.linalg.norm((y[batch] - x_mat[batch].dot(b)))
    bs = method(f_batch_size(batch_size), np.full(k, 1), **config)
    f = f_batch_size(x.shape[0])
    print(f'came close by {f(*bs[-1])}')
    # ax = plt.figure().add_subplot()
    # X = np.arange(len(bs))
    # ax.plot(X, np.vectorize(f)(*bs.T))
    # ax.grid()
    return bs

In [85]:
line = random_perfect_line(10, 100, 10, 10)

In [86]:
def grad(f):
    def grad_help(*args):
        h = 1e-10
        dim = len(args)
        return [(
                        f(*[args[j] + (h if j == i else 0) for j in range(dim)])
                        -
                        f(*[args[j] - (h if j == i else 0) for j in range(dim)])
                ) / (2 * h)
                for i in range(dim)]
    return grad_help

In [87]:
def my_lbfgs(f, x0, m = 5, eps = 1e-9, epochs=100, h_init=1, **config):
    return lbfgs(f, x0, grad(f), m = 5, eps = 1e-9, epochs=100, h_init=1, **config)

In [90]:
regression(*line, my_lbfgs)

ValueError: shapes (10,10) and (1,10) not aligned: 10 (dim 1) != 1 (dim 0)